In [5]:
!pip install -Uq langchain langchain_openai langchain_community youtube-transcript-api

In [6]:
import os
import textwrap
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [7]:
os.environ["OPENAI_API_KEY"] = "..."
try:
    from google.colab import userdata

    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    if not os.environ["OPENAI_API_KEY"]:
        raise ValueError("Colab의 보안 비밀 설정에서 'OPENAI_API_KEY'를 찾을 수 없습니다.")
except ImportError:
    print("Colab 환경이 아니므로, 'api_key' 변수에 직접 값을 할당해야 합니다.")

Colab 환경이 아니므로, 'api_key' 변수에 직접 값을 할당해야 합니다.


In [8]:
video_url = "https://www.youtube.com/watch?v=8KkKuTCFvzI&ab_channel=TED"

In [9]:
video_id = video_url.split("v=")[-1]
try:
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id,
                                                          languages=['en'])
    english_text = " ".join([entry['text'] for entry in transcript_list])
    print(english_text[:1000])
except Exception as e:
    print(f"자막을 가져오는 중 오류 발생: {e}")
    english_text = ""

자막을 가져오는 중 오류 발생: type object 'YouTubeTranscriptApi' has no attribute 'get_transcript'


In [10]:
llm = ChatOpenAI(model="gpt-4o-mini")
summary_prompt_template = """
전체 내용을 검토하고, 4문장 길이의 영어로 요약한 다음 한국어, 러시아어, 독일어로 번역해 주세요.
요약이 내용과 일치하는지 확인하세요.
답변의 영어와 한국어 부분 사이에 '\n----\n' 문자열을 넣어주세요.
답변의 한국어와 러시아어 부분 사이에 '\n****\n' 문자열을 넣어주세요.
답변의 러시아와 독일어 부분 사이에 '\n====\n' 문자열을 넣어주세요.
영어 텍스트:
{content}
"""
summary_prompt = PromptTemplate(template=summary_prompt_template,
                                input_variables=["content"])
summary_chain = summary_prompt | llm
prompt_input = summary_prompt.format(content=english_text)
original_answer = summary_chain.invoke(prompt_input)
print(original_answer.content)
print("\n요약 및 번역 결과:\n")
print(textwrap.fill(
    str(original_answer.content), width=50, replace_whitespace=True
).replace("\\n ", "\n\n"
          ).replace("----", "\n한국어:\n"
                    ).replace("****", "\n러시아어:\n").replace("====", "\n독일어:\n"))

The document outlines the necessary steps to review and summarize content into four sentences. It emphasizes the importance of ensuring the summary accurately reflects the original material. Additionally, it specifies the translation of the summary into Korean, Russian, and German. The format for separating the language sections is clearly defined.

----
문서는 내용을 검토하고 네 문장으로 요약하는 데 필요한 단계들을 설명합니다. 이 요약이 원래 자료를 정확하게 반영해야 한다는 중요성을 강조합니다. 또한 요약을 한국어, 러시아어, 독일어로 번역할 것을 명시합니다. 언어 섹션을 구분하는 형식이 명확히 정의되어 있습니다.

****
Документ описывает необходимые шаги для проверки и резюмирования содержания в четырех предложениях. Подчеркивается важность того, чтобы резюме точно отражало оригинальный материал. Кроме того, указывается на перевод резюме на корейский, русский и немецкий. Формат для разделения языковых секций четко определен.

====
Das Dokument beschreibt die notwendigen Schritte zur Überprüfung und Zusammenfassung des Inhalts in vier Sätzen. Es wird betont, wie wichtig es ist, dass die Zusammenfas

In [11]:
key_points_prompt_template = """
당신은 간결하고 명확한 요점을 제공하는 전문가입니다.
이전 질문에 대한 답변은 다음과 같습니다: <{original_answer}>
이 한국어 답변을 3-5개의 주요 요점으로 정리해 주세요.
각 요점은 글머리 기호로 시작하는 한국어로만 작성해 주세요.
"""
key_points_prompt = PromptTemplate(template=key_points_prompt_template,
                                   input_variables=["original_answer"])
key_points_chain = key_points_prompt | llm
key_points = key_points_chain.invoke({"original_answer": original_answer})
print("\n한국어 주요 요점:\n")
print(key_points.content)


한국어 주요 요점:

- 문서는 내용을 네 문장으로 요약하는 절차를 설명합니다.
- 요약은 원래 자료를 정확하게 반영해야 합니다.
- 요약을 한국어, 러시아어, 독일어로 번역할 것을 명시합니다.
- 언어 섹션을 구분하는 형식이 명확하게 정의되어 있습니다.
